In [ ]:
import numpy as np
from scipy.interpolate import interp1d
import pandas
import math
import matplotlib.pyplot as plt
from matplotlib import ticker
import os

In [ ]:
# set the input parameters
eta_QD = 1
eta_EMF = 2
eta_SSET = 1

mu_Singlet_Sensitiser = 4000 # attenuation coefficient base e (cm-1)

solution_conc_TIPS_Tc = 200 # mg/mL
solution_conc_QD = 100 # mg/mL
mass_quotient = solution_conc_QD/solution_conc_TIPS_Tc

solution_to_solid_ratio = 140 # convert solution mass attenutation coeff to solid state attenutation coeff

def calc_f(solution_conc_QD, solution_conc_TIPS_Tc):
    """ calculate the QD mass fraction from the precurser solution concentrations
    args: 
    solution_conc_QD = the solution conc for the QD
    solution_conc_TIPS_Tc = the solution conc for the TIPS-Tc
    outputs:
    the QD mass fraction
    """
    ans = solution_conc_QD/(solution_conc_QD + solution_conc_TIPS_Tc)
    return ans

f = calc_f(solution_conc_QD, solution_conc_TIPS_Tc)

In [ ]:
# load absorption data

def load_absorption_data(data_path, ratio):
    df = pandas.read_csv(data_path)
    df['ratio'] = ratio
    df['Attenuation Coeff base 10 (cm-1)'] = \
        df['Mass Attenuation Coeff (Lg-1cm-1)']*df['ratio']
    df['Attenuation Coeff (cm-1)'] = \
        df['Attenuation Coeff base 10 (cm-1)']*np.log(10)
    return df

def load_absorption_test():
    TIPS_Tc_data_path = 'Experimental_Data\Mass Attenutation Coeffs TIPS-Tc.csv'
    TIPS_Tc_data = pandas.read_csv(TIPS_Tc_data_path)
    TIPS_Tc_data['ratio']=solution_to_solid_ratio
    TIPS_Tc_data['Attenuation Coeff base 10 (cm-1)'] = TIPS_Tc_data['Mass Attenuation Coeff (Lg-1cm-1)']*TIPS_Tc_data['ratio']
    TIPS_Tc_data['Attenuation Coeff (cm-1)'] = TIPS_Tc_data['Attenuation Coeff base 10 (cm-1)']*np.log(10)
    test_value = np.array(TIPS_Tc_data[:10])
    value = np.array(load_absorption_data(TIPS_Tc_data_path, solution_to_solid_ratio)[:10])
    assert np.all((test_value - value)==0)

TIPS_Tc_data_path = 'Experimental_Data\Mass Attenutation Coeffs TIPS-Tc.csv'
TIPS_Tc_data = load_absorption_data(TIPS_Tc_data_path, solution_to_solid_ratio)

QD_data_path = 'Experimental_Data\Mass Attenutation Coeffs PbS-TET-CA.csv'
QD_data = load_absorption_data(QD_data_path, solution_to_solid_ratio*mass_quotient)

TIPS_An_data_path = 'Experimental_Data\Absorbance TIPS-An.csv'
TIPS_An_data = pandas.read_csv(TIPS_An_data_path)

In [ ]:
# set the varaiables
wl = np.array(QD_data['Wavelength (nm)'])


In [ ]:
# extraction absorption ratios
mu_SF = np.array(TIPS_Tc_data['Attenuation Coeff (cm-1)'])
mu_QD = np.array(QD_data['Attenuation Coeff (cm-1)'])
mu_Sen = mu_Singlet_Sensitiser * np.array(TIPS_An_data['Mass Attenuation Coeff (Lg-1cm-1)'])
mu_total = mu_SF + mu_QD + mu_Sen
alpha_SF = mu_SF/mu_total
alpha_QD = mu_QD/mu_total
alpha_Sen = mu_Sen/mu_total

In [ ]:
def Calc_eta_PM(eta_QD,eta_SSET, eta_EMF, alpha_QD, alpha_SF, alpha_Sen):
    """ calculates the wavelength dependent eta_PM, the SF_PM PLQE
    """
    return eta_QD*( alpha_QD + (alpha_SF + alpha_Sen*eta_SSET)*eta_EMF )

def Calc_eta_PM_test1():
    # no sf absorption condition
    test_alpha_QD = np.ones(100)
    test_alpha_SF = np.zeros(100)
    test_eta_QD = 1
    test_eta_EMF = 2
    assert Calc_eta_PM(test_eta_QD, test_alpha_QD,test_alpha_SF,test_eta_EMF) == np.ones(100)

def Calc_eta_PM_test1():
    # no QD absorption condition
    test_alpha_QD = np.zeros(100)
    test_alpha_SF = np.ones(100)
    test_eta_QD = 1
    test_eta_EMF = 2
    assert Calc_eta_PM(test_eta_QD, test_alpha_QD,test_alpha_SF,test_eta_EMF) == 2*np.ones(100)



In [ ]:
# calculate the PLQE of the SFPM
eta_PM = Calc_eta_PM(eta_QD,eta_SSET, eta_EMF, alpha_QD, alpha_SF, alpha_Sen)
eta_PM_no_EMF = Calc_eta_PM(eta_QD,eta_SSET, 0 , alpha_QD, alpha_SF, alpha_Sen)
eta_PM_no_SSET = Calc_eta_PM(eta_QD, 0 , eta_EMF , alpha_QD, alpha_SF, alpha_Sen)

In [ ]:
# plot the PLQE vs Wavelength

Blue = '#046DE0'
Yellow = '#FDBE3D'
Green = '#38B99E'
Dark_Grey = '#515151'


fig, ax = plt.subplots(1,1)
ax.plot(wl,eta_PM, color = Green,ls = "-",lw=5)
ax.plot(wl,eta_PM_no_SSET, color = Yellow,ls = "-",lw=5)
ax.plot(wl,eta_PM_no_EMF, color = Dark_Grey,ls = "-",lw=5)

ax.set_xlim(400,650)
ax.set_ylim(0,2)
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel(r'$\eta_{PM}$')
yticker = ticker.MultipleLocator(0.5)
ax.yaxis.set_major_locator(yticker)
xticker = ticker.MultipleLocator(100)
ax.xaxis.set_major_locator(xticker)

In [ ]:
# extraction absorption ratios
mu_SF = np.array(TIPS_Tc_data['Attenuation Coeff (cm-1)'])
mu_QD = np.array(QD_data['Attenuation Coeff (cm-1)'])
mu_Sen = np.array(TIPS_An_data['Mass Attenuation Coeff (Lg-1cm-1)'])
n = len(mu_SF)

# amplitude varaiable for the sensitiser
min_log10_amp = -5
max_log10_amp = 7
step_log10_amp = 0.1
n_steps_amp = (max_log10_amp-min_log10_amp)/step_log10_amp
amp = 10**np.linspace(min_log10_amp,max_log10_amp,n_steps_amp+1)
m = len(amp)

# set up the mu matrices
mu_SF = np.reshape(mu_SF,(n,1)) @ np.ones((1,m))
mu_QD = np.reshape(mu_QD,(n,1)) @ np.ones((1,m))
mu_Sen = np.reshape(mu_Sen,(n,1)) @ np.reshape(amp,(1,m))

# calculate the total mu and the fraction absorptions
mu_total = mu_SF + mu_QD + mu_Sen
alpha_SF = mu_SF/mu_total
alpha_QD = mu_QD/mu_total
alpha_Sen = mu_Sen/mu_total


In [ ]:
# calculate the PLQE of the SFPM
eta_PM = Calc_eta_PM(eta_QD,eta_SSET, eta_EMF, alpha_QD, alpha_SF, alpha_Sen)

In [ ]:
# plot the PLQE vs Wavelength

fig, ax = plt.subplots(1,1)
pcm = ax.pcolormesh(wl,alpha_Sen[-1,:],eta_PM.T,\
    vmin = 0, vmax = 2\
    )
fig.colorbar(pcm,ax=ax)
#ax.set_yscale('log')

ax.set_xlim(400,650)
ax.set_ylim(0,1)

ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel(r'$\alpha_{Sen}$')
#yticker = ticker.MultipleLocator(0.5)
#ax.yaxis.set_major_locator(yticker)
xticker = ticker.MultipleLocator(100)
ax.xaxis.set_major_locator(xticker)

In [ ]:
# investigate a particular wavelength
ind =  1167 # 422 nm
ind = 1184 # 405 nm
print('displaying the PM efficiency at ',wl[ind],' nm.')

x=alpha_Sen[ind,:]
y=eta_PM[ind,:]

fig, ax = plt.subplots(1,1)
ax.plot(x,y, color = Green,ls = "-",lw=5)

ax.set_xlim(0,1)
ax.set_ylim(0,2)
ax.set_xlabel(r'$\alpha_{Sen}$')
ax.set_ylabel(r'$\eta_{PM}$')
yticker = ticker.MultipleLocator(0.5)
ax.yaxis.set_major_locator(yticker)
xticker = ticker.MultipleLocator(0.2)
ax.xaxis.set_major_locator(xticker)

In [ ]:
# investigate a particular wavelength
ind =  1167 # 422 nm
ind = 1184 # 405 nm
print('displaying the PM efficiency at ',wl[ind],' nm.')

x=mu_Sen[ind,:]
y=eta_PM[ind,:]

fig, ax = plt.subplots(1,1)
ax.plot(x,y, color = Green,ls = "-",lw=5)

ax.set_xscale('log')
ax.set_xlim(10,1000000)
ax.set_ylim(0,2)
ax.set_xlabel(r'$\mu_{Sen}$')
ax.set_ylabel(r'$\eta_{PM}$')
yticker = ticker.MultipleLocator(0.5)
ax.yaxis.set_major_locator(yticker)
#xticker = ticker.MultipleLocator(0.2)
#ax.xaxis.set_major_locator(xticker)

In [ ]:
# optimal PM performance
# 2 x E_gap(Si) = 2.2 ev ~ 563.6 nm
PM_cutoff = 563.6
mu_SF_optimal = np.array(wl<=PM_cutoff)
mu_QD_optimal = np.array(wl>PM_cutoff)
mu_Sen_optimal = np.zeros(np.shape(mu_SF_optimal))
mu_total_optimal = mu_SF_optimal + mu_QD_optimal + mu_Sen_optimal
alpha_SF_optimal = mu_SF_optimal/mu_total_optimal
alpha_QD_optimal = mu_QD_optimal/mu_total_optimal
alpha_Sen_optimal = mu_Sen_optimal/mu_total_optimal
eta_PM_optimal = Calc_eta_PM(eta_QD,eta_SSET, eta_EMF, alpha_QD_optimal, alpha_SF_optimal, alpha_Sen_optimal)

In [ ]:
# compare with and without reasonable Sen 
# find the peak of the sf attenuation
max_mu_SF = np.max(mu_SF,axis = 0)
 
# find the index of closest value in amp
arg_min = np.argmin(np.abs(amp-max_mu_SF))

fig, ax = plt.subplots(1,1)

# eta_PM = 2 guide
ax.plot([300,650],[2,2], color = '0.6', ls = '--',lw=5)
# eta_PM = 1 guide
ax.plot([300,650],[1,1], color = '0.8', ls = '--',lw=5)

ax.fill_between(wl,eta_PM_optimal,eta_PM[:, arg_min],color = Dark_Grey,alpha = 0.3)
ax.plot(wl,eta_PM_optimal, color = Dark_Grey,ls = "-",lw=5)


ax.fill_between(wl,eta_PM[:, 0],eta_PM[:, arg_min],color = Green,alpha = 0.5)
ax.plot(wl,eta_PM[:, 0], color = Yellow,ls = (1,(3,1)),lw=5)
ax.plot(wl,eta_PM[:, arg_min], color = Green,ls = "-",lw=5)

ax.set_xlim(400,650)
ax.set_ylim(0,2.2)
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel(r'$\eta_{PM}$')
yticker = ticker.MultipleLocator(0.5)
ax.yaxis.set_major_locator(yticker)
xticker = ticker.MultipleLocator(100)
ax.xaxis.set_major_locator(xticker)

In [ ]:
Solar_data_path = 'Experimental_Data\AM1p5G solar spectrum.csv'
Solar_data = pandas.read_csv(Solar_data_path)
Solar_spec = np.array(Solar_data['photon flux (s-1 m-2 nm-1)'])
wl_spec = np.array(Solar_data['Wvlgth nm'])
Solar_interp = interp1d(wl_spec,Solar_spec)
solar = np.reshape(np.array(Solar_interp(wl)),(n,1))


In [ ]:
# clip by the Si bandgap
Si_cutoff = 2*563.6
Abs_Si = np.reshape(np.array(wl<Si_cutoff),(n,1))

I_lambda = Abs_Si*solar
I_lambda_optimal = np.reshape(eta_PM_optimal,(n,1))*Abs_Si*solar
I_lambda_no_sen = eta_PM[:, [0]]*Abs_Si*solar
I_lambda_sen = eta_PM[:, [arg_min]]*Abs_Si*solar

fig, ax = plt.subplots(1,1)
ax.plot(wl,solar, color = '0.8', ls = '-' ,lw=5)
ax.plot(wl,I_lambda, color = '0.6', ls = '-',lw=5)

ax.fill_between(wl,I_lambda_optimal[:,0],I_lambda_sen[:,0],color = Dark_Grey,alpha = 0.3)
ax.plot(wl,I_lambda_optimal, color = Dark_Grey,ls = "-",lw=5)

ax.fill_between(wl,I_lambda_no_sen[:,0],I_lambda_sen[:,0],color = Green,alpha = 0.5)
ax.plot(wl,I_lambda_no_sen, color = Yellow,ls = '-',lw=5)
ax.plot(wl,I_lambda_sen, color = Green,ls = "-",lw=5)

ax.set_xlim(400,1600)
ax.set_ylim(0, 9E18)
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel(r'$Photons per sec per m^2$')
#yticker = ticker.MultipleLocator(0.5)
#ax.yaxis.set_major_locator(yticker)
xticker = ticker.MultipleLocator(100)
ax.xaxis.set_major_locator(xticker)

In [ ]:
def integrate_spec(x, spec):
    dx = np.abs(x[1]-x[0])
    return np.sum(spec,axis = 0)*dx

I = integrate_spec(wl,I_lambda)
I_optimal = integrate_spec(wl,I_lambda_optimal)
I_no_sen = integrate_spec(wl,I_lambda_no_sen)
I_sen = integrate_spec(wl,I_lambda_sen)

print(I/I,'\n',I_optimal/I,'\n',I_no_sen/I,'\n',I_sen/I)